In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from lifetimes.utils import summary_data_from_transaction_data
import numpy as np
import datetime
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from lifetimes import ModifiedBetaGeoFitter
from lifetimes import GammaGammaFitter
import dask.dataframe as dd
from dask.multiprocessing import get
from collections import Counter
from sklearn.model_selection import train_test_split
import gc
import joblib
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
config = tf.ConfigProto(device_count={"CPU": 20})
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dropout
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV, KFold
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import layers
from keras import Input
from keras.layers import BatchNormalization
from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import expon
from keras.models import load_model
import io
import json
from keras_preprocessing.text import tokenizer_from_json
from gensim.models.doc2vec import Doc2Vec

Using TensorFlow backend.


In [53]:
def _write_dataframe_to_csv_on_s3(df, path_s3):
    import boto3
    from io import StringIO
    """ Write a dataframe to a CSV on S3 """
    a = path_s3.split('//')
    b = a[1].split('/')
    bucket = b[0]
    c = path_s3.split(bucket+'/')
    path = c[1]
    
    buffer = StringIO()
    df.to_csv(buffer,index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, path).put(Body=buffer.getvalue())
    return None

In [60]:
visits = pd.read_csv("s3://fda-labs/ltv-ml/ML/summary_visitas_train.csv")

s3 = boto3.client('s3')

for name in ["doc2vec_20_20","doc2vec_10_20","doc2vec_20_12","doc2vec_20_8","doc2vec_32_20"]:
    
    s3.download_file("fda-labs", "ltv-ml/Embedding/"+name+"/"+name,name)
    s3.download_file("fda-labs", "ltv-ml/Embedding/"+name+"/"+name+".docvecs.vectors_docs.npy",
                     name+".docvecs.vectors_docs.npy")

    model = Doc2Vec.load(name)

    # aplico vectorizacion
    vector = dd.from_pandas(visits, npartitions=cpu_count()).apply(
        lambda x: model.infer_vector(x.phrase.split(' ')),axis = 1).compute(scheduler='processes')

    # trsnformo el resultado en df
    vector = pd.DataFrame.from_records(np.array(vector)).head()
    vector.columns = [name+"_"+str(x) for x in list(vector.columns)]

    # le agrego el customer y guardo
    vector["cust"] = visits["cust"]
    _write_dataframe_to_csv_on_s3(vector,"s3://fda-labs/ltv-ml/Embedding/preproc_Doc2vec/train_"+name+".csv")

/usr/local/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.5/site-packages/dask/dataframe/core.py:3966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead

In [ ]:
visits = pd.read_csv("s3://fda-labs/ltv-ml/ML/summary_visitas_test.csv")

s3 = boto3.client('s3')

for name in ["doc2vec_20_20","doc2vec_10_20","doc2vec_20_12","doc2vec_20_8","doc2vec_32_20"]:
    
    s3.download_file("fda-labs", "ltv-ml/Embedding/"+name+"/"+name,name)
    s3.download_file("fda-labs", "ltv-ml/Embedding/"+name+"/"+name+".docvecs.vectors_docs.npy",
                     name+".docvecs.vectors_docs.npy")

    model = Doc2Vec.load(name)

    # aplico vectorizacion
    vector = dd.from_pandas(visits, npartitions=cpu_count()).apply(
        lambda x: model.infer_vector(x.phrase.split(' ')),axis = 1).compute(scheduler='processes')

    # trsnformo el resultado en df
    vector = pd.DataFrame.from_records(np.array(vector)).head()
    vector.columns = [name+"_"+str(x) for x in list(vector.columns)]

    # le agrego el customer y guardo
    vector["cust"] = visits["cust"]
    _write_dataframe_to_csv_on_s3(vector,"s3://fda-labs/ltv-ml/Embedding/preproc_Doc2vec/test_"+name+".csv")